# Lecture 7: Rows, Columns, Images

Two dimensional data arrays are normally stored in column-major or row-major order. 

In **row-major**, order adjacent elements within a **row** are stored next to each other in memory. In **column-major**, order adjacent elements within a **column** are stored next to each other in memory. See also https://en.wikipedia.org/wiki/Matrix_representation.

For the usual mathematical matrix notation $A_{ij}$, where $i$ is the row, and $j$ the column, we have in the case of a $3x4$ matrix:

$$
A = \begin{bmatrix}
a_{11} & a_{12} & a_{13} & a_{14}\\
a_{21} & a_{22} & a_{23} & a_{24}\\
a_{31} & a_{32} & a_{33} & a_{34}\\
\end{bmatrix}
$$

or (zero-indexed):

$$
A = \begin{bmatrix}
a_{00} & a_{01} & a_{02} & a_{03}\\
a_{10} & a_{11} & a_{12} & a_{13}\\
a_{20} & a_{21} & a_{33} & a_{23}\\
\end{bmatrix}
$$

Classic languages such as Fortran store their arrays in so-called column-major order. **FDATA(NR,NC)**, and indices start at 1. 

More modern language, such a C, store their arrays in row-major order,  **CDATA[NR][NC]**, with indices starting at 0.
```
    col major:   fdata(1,1),  fdata(2,1), ...        first index runs fastest 
    row major:   cdata[0][0], cdata[0][1], ...       last index runs fastest  
```

Examples of column major are: Fortran, MatLab, IDL, R, and Julia

Examples of row major are: C and Python

Images or matrices are often referred to in X and Y coordinates, like a mathematical system. The origin would be at (0,0) in the lower left corner; however, this convention can vary from field to field. For instance, image processing software normally puts the (0,0) origin at the top left corner, which is similar to how the matrix above is printed. 

This, together with row-major and column-major can make it challenging to interchange data and plot them on the screen between fields and programming language and having to re-order axis' (especially for large data sets) can be a very expensive operation.

## 1. A Simple Example


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

# lets import fits from astropy,
#  one of many more complex file types
#  for storing multi-dimensional data
# common data type for astronomy
# Flexible Image Transport System
# Other's include, CDF, Net CDF, HD5, matlab .m files, idl .sav files

from astropy.io import fits         

Get the Header-Data-Units (hdu's) from a fits file. This particular one only has 1.

In [ ]:
# open a simple fit's file

hdu = fits.open('../data/cube432.fits')
print(len(hdu))
hdu.info()

In [ ]:
# access the header
h = hdu[0].header
# access the data
d = hdu[0].data

#what is the header?
print(type(h))

#what type of data?
print(type(d))
print(d.shape)
print(d)

In [ ]:
f=d[1,:,:]
print(f)

f[:,1:3] #slicing does not include the last index

This 4x3x2 matrix can actually also be generated from scratch using basic numpy:

In [ ]:


d1 = np.zeros(2*3*4).reshape(2,3,4)
dimensions = d1.shape
for z in range(dimensions[0]):
    for y in range(dimensions[1]):
        for x in range(dimensions[2]):
            d1[z,y,x] = x + 10*y + 100*z
print(d1)


d1.shape

In [ ]:
np.arange(2*3*4).reshape((2,3,4))

In [ ]:
print(d1.flatten())

In [ ]:
print(d1.flatten('f'))

In [ ]:
# are two arrays the same (or close enough?)
print(d)
print(d1)
np.allclose(d,d1) #Returns True if two arrays are element-wise equal within a tolerance.

## 2. Slicing Multi-Dimensional Arrays/Images

For a 3-D array, you can think of things being indexed as [z,y,x]. 

This 3-D array could be multiple images with the z-axis corresponding to time or a each of z, y, and x corresponding to spatial coordinates. 

The 2x3x4 matrix we loaded above would look like:

In [ ]:
d1 = np.zeros(2*3*4).reshape(2,3,4)
for z in range(2):
    for y in range(3):
        for x in range(4):
            d1[z,y,x] = x + 10*y + 100*z
print(d1)


In [ ]:
print('--')
print(d1[1])
print('--')
print(d1[1,0,0:2])
print('--')
print(d1[1,0:2,0])
print('--')
print(d1[1,0:2,1::])
print('--')
print(d1[1,1:3,1:-1])
print('--')
print(d1[0,:,1:])

## 3. Manipulating Dimensions/Axes of Multi-Dimensional Arrays/Images 

We now want to take a plane from this cube, and plot this in a heatmap or contour map. We are now faced deciding how columns and rows translate to X and Y on a plot. Math, Astronomy, Geography and Image Processing groups all differ a bit how they prefer to see this, so numpy comes  with a number of function to help you with this:

* np.reshape
* np.transpose (or T)
* np.flipud
* np.fliplr
* np.rot90
* np.swapaxes
* np.moveaxis

the important thing to realize is that they all give a new **view** of the array, which often is more efficient as moving the actual values.

In [ ]:
p0 = d[0,:,:]
p1 = d[1,:,:]
print(p0)
plt.imshow(p0)
plt.colorbar()

In [ ]:
plt.matshow(p0,origin='lower')

Note that for a small 4x3 matrix this image has been artificially made smooth by interpolating in imshow(); however you can already see that the integer coordinates are at the center of a cell:  (0.0) is the center of the lower left cell. This is a little more obvious when you turn off interpolation:


In [ ]:
plt.imshow(p0,interpolation='none',origin='lower')
plt.colorbar()

if you want to print the array values on the terminal with 0 at the bottom left, use the np.flipup() function:

In [ ]:
print(np.flipud(p0))
plt.imshow(np.flipud(p0),interpolation='none')
plt.colorbar()

In [ ]:
import numpy.ma as ma

a = np.arange(4)
am = ma.masked_equal(a,2)
print(a.sum(),am.sum())
print(am.data,am.mask)

### Some Real Data


In [ ]:
# lets load the matplot lib color scales
#  and image module to readin images

import matplotlib.colors as colors
import matplotlib.image as mpimg

hdu = fits.open('../data/PKR_DASC_0558_20151220_044655.730.FITS')

In [ ]:
print(len(hdu))
for HDU in hdu:
    print(HDU)

In [ ]:
h = hdu[0].header
d = hdu[0].data
print(d.shape, d.min(), d.max(), d.mean(), np.median(d), d.std())
hdu.info()

In [ ]:
im = d
plt.imshow(im,origin=['Lower'], norm=colors.LogNorm(vmin=300,vmax=1000) )
plt.colorbar()
print(im.shape)

In [ ]:
plt.imshow(np.rot90(im,k=1), origin=['Lower'], norm=colors.LogNorm(vmin=300,vmax=2000), cmap='Greens' )
plt.colorbar()

In [ ]:
img = mpimg.imread('../data/20151220_081842_yknf_slr-08_full.jpg')
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
# Plot the different RGB channels
fig, axes = plt.subplots(1, 4, figsize=(20,12))

axes[0].imshow(img[:, :, 0], cmap='Reds')
axes[1].imshow(img[:, :, 1], cmap='Greens') 
axes[2].imshow(img[:, :, 2], cmap='Blues') 
axes[3].imshow(img)

for a in axes:
    a.set_xticklabels([])
    a.set_yticklabels([])

## 4. Contour Plots

Ideal for plotting multple Image-like datasets on the same figure, such as identifying active regions on the sun (etc. coronal holes, sun spots) or the crustal magnetic field strength at Mars (below).

![Mars Crustal Fields](../data/MarsCrustalFields.png "Mars Crustal Fields")
[The construction of sparse models of Mars's crustal magnetic field [Moore and Bloxham 2017]](http://onlinelibrary.wiley.com.proxy-um.researchport.umd.edu/doi/10.1002/2016JE005238/full). 

The plot above was generated in **exercises/MarsCrustalField.ipynb**.


In [ ]:
def f(x,y):
    return (1-x/2+x**5+y**3)*np.exp(-x**2-y**2)

n = 256
x = np.linspace(-3,3,n)
y = np.linspace(-3,3,n)
X,Y = np.meshgrid(x,y)

plt.axes([0.025,0.025,0.95,0.95])

plt.contourf(X, Y, f(X,Y), 10, alpha=.75, cmap=plt.cm.hot)
plt.colorbar()

C = plt.contour(X, Y, f(X,Y), 10, colors='black', linewidths=.5)
plt.clabel(C, inline=1, fontsize=10)
plt.xticks([])
plt.yticks([])


## 5. Matrix Manipulation



### Inner and Outer loop order of execution

Set up a (random) square matrix and vector. Multiply the matrix with a vector and measure the performance difference if you order the loops differently.

In [ ]:
%%time 
n = 1000
m = n*10
np.random.seed(123)
a = np.random.normal(size=m*n).reshape(m,n)
x = np.random.normal(size=n)
print(x[0])
#
#a = np.arange(n*n).reshape(n,n)
#x = np.arange(n)

In [ ]:
%%timeit
b = np.matmul(a,x)

In [ ]:
print(a.shape,x.shape)

In [ ]:
%%timeit
b1 = np.zeros(m)
for i in range(m):
    for j in range(n):
        b1[i] = b1[i] + a[i,j]*x[j]

In [ ]:
%%timeit
b2 = np.zeros(m)
for i in range(m):
    ai = a[i,:]
    b2[i] = b2[i] + np.inner(ai,x)

In [ ]:
%%timeit
b3 = np.zeros(m)
for j in range(n):
    for i in range(m):
        b3[i] = b3[i] + a[i,j]*x[j]

In [ ]:
if n < 3:
    print('a',a,'\nx',x)
    print('b',b,'\nb1',b1,'\nb2',b2,'\nb3',b3)
else:
    print(n)

### Transposing

In [ ]:
%%timeit
b = a.transpose()
# note B is another view of A

In [ ]:
%%timeit
c = np.zeros(n*m).reshape(n,m)

for j in range(n):
    for i in range(m):
        c[j,i] = a[i,j]

### Inversion

In [ ]:
from numpy.linalg import inv


In [ ]:
# Let's create a simple 2x2 matrix
n = 2
a1 = np.random.normal(size=n*n).reshape(n,n)

print(a1)

In [ ]:
# and calculate the inverse 
ainv = inv(a1)

In [ ]:
print(a1)
print(ainv)

np.matrix(a1) * np.matrix(ainv)

In [ ]:
i1=np.matmul(a1,ainv)

In [ ]:
i0=np.eye(n)

In [ ]:
print(np.allclose(i0,i1,atol=1e-10))
print(i1)